# 6章 pandasを使ったデータ加工

In [3]:
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

%precision 3

'%.3f'

In [5]:
# 階層型インデックス
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [['a','a','b'],
            [1,2,2]],
    columns = [['Osaka','Tokyo','Osaka'],
              ['Blue','Red','Red']]    
)
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [6]:
# indexやcolumnsに名前を付ける
hier_df.index.names = ['key1','key2']
hier_df.columns.names = ['city','color']
hier_df

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [7]:
hier_df['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

In [8]:
# インデックスを軸にした集計
hier_df.sum(level = 'key2',axis = 0)

city,Osaka,Tokyo,Osaka
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


In [9]:
hier_df.sum(level = 'color',axis = 1)

color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

In [10]:
hier_df.drop(['b'])

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

In [11]:
# 練習問題6-1
hier_df1 = DataFrame(
    np.arange(12).reshape((3,4)),
    index = [['c','d','d'],
            [1,2,1]],
    columns = [['Kyoto','Nagoya','Hokkaido','Kyoto'],
              ['Yellow','Yellow','Red','Blue']]    
)
hier_df1.index.names = ['key1','key2']
hier_df1.columns.names = ['city','color']
hier_df1

city       Kyoto Nagoya Hokkaido Kyoto
color     Yellow Yellow      Red  Blue
key1 key2                             
c    1         0      1        2     3
d    2         4      5        6     7
     1         8      9       10    11

In [12]:
hier_df1['Kyoto']

color      Yellow  Blue
key1 key2              
c    1          0     3
d    2          4     7
     1          8    11

In [18]:
hier_df1.mean(level='city',axis=1)

city       Kyoto  Nagoya  Hokkaido
key1 key2                         
c    1       1.5     1.0       2.0
d    2       5.5     5.0       6.0
     1       9.5     9.0      10.0

In [19]:
hier_df1.sum(level='key2',axis=0)

city,Kyoto,Nagoya,Hokkaido,Kyoto
color,Yellow,Yellow,Red,Blue
key2,,,,
1,8,10,12,14
2,4,5,6,7


### データの結合

In [51]:
data1 = {
    'id':['100','101','102','103','104','106','108','110','111','113'],
    'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo'],
    'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981],
    'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']
}
df1 = DataFrame(data1)
df1

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [23]:
data2 = {
    'id':['100','101','102','105','107'],
    'math':[50,43,33,76,98],
    'english':[90,30,20,50,30],
    'sex':['M','F','F','M','M'],
    'index_num':[0,1,2,3,4]
}
df2 = DataFrame(data2)
df2

,id,math,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


In [24]:
# データのマージ(全結合)
pd.merge(df1,df2,on='id')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50,90,M,0
1,101,Osaka,1989,Akiko,43,30,F,1
2,102,Kyoto,1992,Yuki,33,20,F,2


In [25]:
# 全結合
pd.merge(df1,df2,how = 'outer')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN


In [26]:
# indexによるマージ
pd.merge(df1,df2,left_index = True,right_on = 'index_num')


,id_x,city,birth_year,name,id_y,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,100,50,90,M,0
1,101,Osaka,1989,Akiko,101,43,30,F,1
2,102,Kyoto,1992,Yuki,102,33,20,F,2
3,103,Hokkaido,1997,Satoru,105,76,50,M,3
4,104,Tokyo,1982,Steeve,107,98,30,M,4


In [27]:
# 左外部結合
pd.merge(df1,df2,how = 'left')

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN


In [28]:
data3 = {
    'id':['117','118','119','120','125'],
    'city':['Chiba','Kanagawa','Tokyo','Fukuoka','Okinawa'],
    'birth_year':[1990,1989,1992,1997,1982],
    'name':['Suguru','Kouichi','Satochi','Yukie','Akari']
}
df3 = DataFrame(data3)
df3

,id,city,birth_year,name
0,117,Chiba,1990,Suguru
1,118,Kanagawa,1989,Kouichi
2,119,Tokyo,1992,Satochi
3,120,Fukuoka,1997,Yukie
4,125,Okinawa,1982,Akari


In [29]:
# concat (縦結合)
concat_data = pd.concat([df1,df3])
concat_data

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [31]:
# ピポット操作
hier_df= DataFrame(
    np.arange(9).reshape((3,3,)),
    index = [
        ['a','a','b'],
        [1,2,2]
    ],
    columns = [
        ['Osaka','Tokyo','Osaka'],
        ['Blue','Red','Red']
    ]

)
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [32]:
hier_df.stack()

Osaka  Tokyo
a 1 Blue      0    NaN
    Red       2    1.0
  2 Blue      3    NaN
    Red       5    4.0
b 2 Blue      6    NaN
    Red       8    7.0

In [33]:
hier_df.stack().unstack()

Osaka     Tokyo     
     Blue Red  Blue  Red
a 1     0   2   NaN  1.0
  2     3   5   NaN  4.0
b 2     6   8   NaN  7.0

In [34]:
#重複データの除去
dupli_data  = DataFrame({
    'col1':[1,1,2,3,4,4,6,6],
    'col2':['a','b','b','b','c','c','b','b']
}
)
print('・元のデータ')
dupli_data

・元のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,b
7,6,b


In [35]:
# 重複判定
dupli_data.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [36]:
dupli_data.drop_duplicates()

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,b


In [37]:
# マッピング処理
city_map = {
    'Tokyo':'Kanto',
    'Hokkaido':'Hokkaido',
    'Osaka':'Kansai',
    'Kyoto':'Kansai'
}
city_map

{'Tokyo': 'Kanto',
 'Hokkaido': 'Hokkaido',
 'Osaka': 'Kansai',
 'Kyoto': 'Kansai'}

In [52]:
# 参照データを結合
df1['region'] = df1['city'].map(city_map)
df1

,id,city,birth_year,name,region
0,100,Tokyo,1990,Hiroshi,Kanto
1,101,Osaka,1989,Akiko,Kansai
2,102,Kyoto,1992,Yuki,Kansai
3,103,Hokkaido,1997,Satoru,Hokkaido
4,104,Tokyo,1982,Steeve,Kanto
5,106,Tokyo,1991,Mituru,Kanto
6,108,Osaka,1988,Aoi,Kansai
7,110,Kyoto,1990,Tarou,Kansai
8,111,Hokkaido,1995,Suguru,Hokkaido
9,113,Tokyo,1981,Mitsuo,Kanto


In [53]:
df1['up_two_num'] = df1['birth_year'].map(lambda x: str(x)[0:3])
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [42]:
# ビン分割
birth_year_bins = [1980,1985,1990,1995,2000]
birth_year_cut_data = pd.cut(df1.birth_year, birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

In [43]:
pd.value_counts(birth_year_cut_data)

(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: birth_year, dtype: int64

In [44]:
group_names = ['early1980s','lata1980s','early1990s','late1990s']
birth_year_cut_data = pd.cut(df1.birth_year,birth_year_bins,labels = group_names)
pd.value_counts(birth_year_cut_data)

lata1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: birth_year, dtype: int64

In [45]:
pd.cut(df1.birth_year,2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

In [46]:
pd.value_counts(pd.qcut(df1.birth_year,2))

(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: birth_year, dtype: int64

In [54]:
# データの集約とグループ演算
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [55]:
df1.groupby('city').size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

In [56]:
df1.groupby('city')['birth_year'].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

In [58]:
df1.groupby(['region','city'])['birth_year'].mean()

region    city    
Hokkaido  Hokkaido    1996.0
Kansai    Kyoto       1991.0
          Osaka       1988.5
Kanto     Tokyo       1986.0
Name: birth_year, dtype: float64

In [59]:
df1.groupby(['region','city'],as_index = False)['birth_year'].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kansai,Osaka,1988.5
3,Kanto,Tokyo,1986.0


In [61]:
for group , subdf in df1.groupby('region'):
    print('==============================================')
    print('Region Name:{0}'.format(group))
    print(subdf)

Region Name:Hokkaido
    id      city  birth_year    name    region up_two_num
3  103  Hokkaido        1997  Satoru  Hokkaido        199
8  111  Hokkaido        1995  Suguru  Hokkaido        199
Region Name:Kansai
    id   city  birth_year   name  region up_two_num
1  101  Osaka        1989  Akiko  Kansai        198
2  102  Kyoto        1992   Yuki  Kansai        199
6  108  Osaka        1988    Aoi  Kansai        198
7  110  Kyoto        1990  Tarou  Kansai        199
Region Name:Kanto
    id   city  birth_year     name region up_two_num
0  100  Tokyo        1990  Hiroshi  Kanto        199
4  104  Tokyo        1982   Steeve  Kanto        198
5  106  Tokyo        1991   Mituru  Kanto        199
9  113  Tokyo        1981   Mitsuo  Kanto        198


In [63]:
student_data_math = pd.read_csv('./chap3/student-mat.csv',sep=';')
functions = ['count','mean','max','min']
grouped_student_math_data1 = student_data_math.groupby(['sex','address'])
grouped_student_math_data1['age','G1'].agg(functions)

<ipython-input-63-d769434e6bbf>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_student_math_data1['age','G1'].agg(functions)


age                       G1                   
            count       mean max min count       mean max min
sex address                                                  
F   R          44  16.977273  19  15    44  10.295455  19   6
    U         164  16.664634  20  15   164  10.707317  18   4
M   R          44  17.113636  21  15    44  10.659091  18   3
    U         143  16.517483  22  15   143  11.405594  19   5

### 欠損データと異常値の取り扱い 

In [65]:
import numpy as np
from numpy import nan as NA
import pandas as pd

df = pd.DataFrame(np.random.rand(10,4))

df.iloc[1,0] = NA
df.iloc[2:3,2] = NA
df.iloc[5:,3] = NA

In [66]:
df

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
1,NaN,0.678109,0.347438,0.644791
2,0.729852,0.959897,NaN,0.987306
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373
5,0.222109,0.543993,0.568119,NaN
6,0.711830,0.678771,0.163992,NaN
7,0.631501,0.750483,0.771915,NaN
8,0.317201,0.432321,0.716561,NaN
9,0.635314,0.263141,0.629660,NaN


In [67]:
df.dropna()

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373


In [68]:
df[[0,1]].dropna()

,0,1
0,0.832051,0.833810
2,0.729852,0.959897
3,0.180015,0.195259
4,0.804926,0.074727
5,0.222109,0.543993
6,0.711830,0.678771
7,0.631501,0.750483
8,0.317201,0.432321
9,0.635314,0.263141


In [69]:
df.fillna(0)

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
1,0.000000,0.678109,0.347438,0.644791
2,0.729852,0.959897,0.000000,0.987306
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373
5,0.222109,0.543993,0.568119,0.000000
6,0.711830,0.678771,0.163992,0.000000
7,0.631501,0.750483,0.771915,0.000000
8,0.317201,0.432321,0.716561,0.000000
9,0.635314,0.263141,0.629660,0.000000


In [70]:
df.fillna(method='ffill')

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
1,0.832051,0.678109,0.347438,0.644791
2,0.729852,0.959897,0.347438,0.987306
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373
5,0.222109,0.543993,0.568119,0.550373
6,0.711830,0.678771,0.163992,0.550373
7,0.631501,0.750483,0.771915,0.550373
8,0.317201,0.432321,0.716561,0.550373
9,0.635314,0.263141,0.629660,0.550373


In [71]:
df.mean()

0    0.562755
1    0.541051
2    0.518955
3    0.569579
dtype: float64

In [72]:
df.fillna(df.mean())

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
1,0.562755,0.678109,0.347438,0.644791
2,0.729852,0.959897,0.518955,0.987306
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373
5,0.222109,0.543993,0.568119,0.569579
6,0.711830,0.678771,0.163992,0.569579
7,0.631501,0.750483,0.771915,0.569579
8,0.317201,0.432321,0.716561,0.569579
9,0.635314,0.263141,0.629660,0.569579


In [73]:
# 練習問題6-13
df2 = pd.DataFrame(np.random.rand(15,6))
df2.iloc[2,0] = NA
df2.iloc[5:9,2] = NA
df2.iloc[7:10,3] = NA
df2.iloc[10,5] = NA

df2

,0,1,2,3,4,5
0,0.185159,0.247804,0.289338,0.831896,0.747366,0.815335
1,0.484923,0.735411,0.889819,0.071061,0.942163,0.924370
2,NaN,0.456950,0.523428,0.503952,0.797866,0.937405
3,0.321699,0.705811,0.870362,0.528252,0.688856,0.393130
4,0.169900,0.024065,0.507174,0.882759,0.657613,0.488186
5,0.136411,0.250220,NaN,0.616450,0.462011,0.640714
6,0.895717,0.815792,NaN,0.192949,0.861277,0.437918
7,0.391439,0.008516,NaN,NaN,0.332481,0.792120
8,0.063374,0.436505,NaN,NaN,0.303649,0.786540
9,0.251011,0.109078,0.503392,NaN,0.040564,0.986071


In [74]:
df2.dropna()

,0,1,2,3,4,5
0,0.185159,0.247804,0.289338,0.831896,0.747366,0.815335
1,0.484923,0.735411,0.889819,0.071061,0.942163,0.924370
3,0.321699,0.705811,0.870362,0.528252,0.688856,0.393130
4,0.169900,0.024065,0.507174,0.882759,0.657613,0.488186
11,0.975568,0.498867,0.781100,0.103601,0.002422,0.762358
12,0.258712,0.925192,0.761883,0.053426,0.900094,0.130045
13,0.455134,0.450709,0.764638,0.592813,0.897003,0.105124
14,0.427627,0.158034,0.480718,0.029191,0.173373,0.036566


In [76]:
df2.fillna(0)

,0,1,2,3,4,5
0,0.185159,0.247804,0.289338,0.831896,0.747366,0.815335
1,0.484923,0.735411,0.889819,0.071061,0.942163,0.924370
2,0.000000,0.456950,0.523428,0.503952,0.797866,0.937405
3,0.321699,0.705811,0.870362,0.528252,0.688856,0.393130
4,0.169900,0.024065,0.507174,0.882759,0.657613,0.488186
5,0.136411,0.250220,0.000000,0.616450,0.462011,0.640714
6,0.895717,0.815792,0.000000,0.192949,0.861277,0.437918
7,0.391439,0.008516,0.000000,0.000000,0.332481,0.792120
8,0.063374,0.436505,0.000000,0.000000,0.303649,0.786540
9,0.251011,0.109078,0.503392,0.000000,0.040564,0.986071


In [77]:
df2.mean()

0    0.405534
1    0.450041
2    0.653572
3    0.368475
4    0.522225
5    0.588277
dtype: float64

In [78]:
df.fillna(df2.mean())

,0,1,2,3
0,0.832051,0.833810,0.486146,0.325228
1,0.405534,0.678109,0.347438,0.644791
2,0.729852,0.959897,0.653572,0.987306
3,0.180015,0.195259,0.311114,0.340199
4,0.804926,0.074727,0.675654,0.550373
5,0.222109,0.543993,0.568119,0.368475
6,0.711830,0.678771,0.163992,0.368475
7,0.631501,0.750483,0.771915,0.368475
8,0.317201,0.432321,0.716561,0.368475
9,0.635314,0.263141,0.629660,0.368475


In [80]:
import pandas_datareader.data as pdr

In [81]:
start_date = '2001/1/2'
end_date = '2016/12/30'
fx_jpusdata = pdr.DataReader('DEXJPUS','fred',start_date,end_date)

In [82]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


In [83]:
fx_jpusdata['2016-04']

,DEXJPUS
DATE,
2016-04-01,112.06
2016-04-04,111.18
2016-04-05,110.26
2016-04-06,109.63
2016-04-07,107.98
2016-04-08,108.36
2016-04-11,107.96
2016-04-12,108.54
2016-04-13,109.21


In [84]:
fx_jpusdata.resample('M').last().head()

,DEXJPUS
DATE,
2001-01-31,116.39
2001-02-28,117.28
2001-03-31,125.54
2001-04-30,123.57
2001-05-31,118.88


In [85]:
fx_jpusdata.resample('D').last().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,NaN


In [86]:
fx_jpusdata.resample('D').ffill().head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-06,116.19


In [87]:
fx_jpusdata.shift(1).head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,114.73
2001-01-04,114.26
2001-01-05,115.47
2001-01-08,116.19


In [88]:
fx_jpusdata_ratio = fx_jpusdata / fx_jpusdata.shift(1)
fx_jpusdata_ratio.head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,0.995903
2001-01-04,1.010590
2001-01-05,1.006235
2001-01-08,0.998107


In [89]:
fx_jpusdata.head()

,DEXJPUS
DATE,
2001-01-02,114.73
2001-01-03,114.26
2001-01-04,115.47
2001-01-05,116.19
2001-01-08,115.97


In [90]:
fx_jpusdata.rolling(3).mean().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,114.820000
2001-01-05,115.306667
2001-01-08,115.876667


In [91]:
fx_jpusdata.rolling(3).std().head()

,DEXJPUS
DATE,
2001-01-02,NaN
2001-01-03,NaN
2001-01-04,0.610000
2001-01-05,0.975312
2001-01-08,0.368963
